In [ ]:
import numpy as np
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque, defaultdict
import os

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Prioritized Experience Replay Buffer
class PrioritizedReplayBuffer:
    def __init__(self, buffer_size, batch_size, alpha=0.6):
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.alpha = alpha
        self.buffer = []
        self.pos = 0
        self.priorities = np.zeros((buffer_size,), dtype=np.float32)

    def add(self, state, action, reward, next_state, done):
        max_prio = self.priorities.max() if self.buffer else 1.0
        if len(self.buffer) < self.buffer_size:
            self.buffer.append((state, action, reward, next_state, done))
        else:
            self.buffer[self.pos] = (state, action, reward, next_state, done)
        self.priorities[self.pos] = max_prio
        self.pos = (self.pos + 1) % self.buffer_size

    def sample(self, beta=0.4):
        if len(self.buffer) == self.buffer_size:
            prios = self.priorities
        else:
            prios = self.priorities[:self.pos]
        probs = prios ** self.alpha
        probs /= probs.sum()
        indices = np.random.choice(len(self.buffer), self.batch_size, p=probs)
        samples = [self.buffer[idx] for idx in indices]
        total = len(self.buffer)
        weights = (total * probs[indices]) ** (-beta)
        weights /= weights.max()
        batch = list(zip(*samples))
        states, actions, rewards, next_states, dones = batch
        return (
            torch.FloatTensor(np.array(states)).to(device),
            torch.LongTensor(actions).to(device),
            torch.FloatTensor(rewards).to(device),
            torch.FloatTensor(np.array(next_states)).to(device),
            torch.FloatTensor(dones).to(device),
            indices,
            torch.FloatTensor(weights).to(device)
        )

    def update_priorities(self, indices, priorities):
        for idx, prio in zip(indices, priorities):
            self.priorities[idx] = prio

    def __len__(self):
        return len(self.buffer)

# Dueling Q-Network
class DuelingQNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(DuelingQNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.value_stream = nn.Linear(64, 1)
        self.advantage_stream = nn.Linear(64, action_size)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        value = self.value_stream(x)
        advantage = self.advantage_stream(x)
        q_vals = value + (advantage - advantage.mean())
        return q_vals

# Independent Q-Learning Agent
class IQLAgent:
    def __init__(self, state_size, action_size, agent_id, lr=3e-4, gamma=0.99, tau=1e-3,
                 buffer_size=100000, batch_size=64, alpha=0.6, beta_start=0.4, beta_frames=100000):
        self.agent_id = agent_id
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.tau = tau
        self.batch_size = batch_size

        self.q_network = DuelingQNetwork(state_size, action_size).to(device)
        self.target_network = DuelingQNetwork(state_size, action_size).to(device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()

        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        self.replay_buffer = PrioritizedReplayBuffer(buffer_size, batch_size)

        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999

        self.beta = beta_start
        self.beta_increment = (1.0 - beta_start) / beta_frames

    def choose_action(self, state, evaluate=False):
        if evaluate or random.random() > self.epsilon:
            state = torch.FloatTensor(state).unsqueeze(0).to(device)
            with torch.no_grad():
                q_values = self.q_network(state)
            action = torch.argmax(q_values).item()
        else:
            action = random.randrange(self.action_size)
        return action

    def step(self, state, action, reward, next_state, done):
        self.replay_buffer.add(state, action, reward, next_state, done)
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

        for _ in range(2):
            self.learn()

        self.beta = min(1.0, self.beta + self.beta_increment)

    def learn(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        states, actions, rewards, next_states, dones, indices, weights = self.replay_buffer.sample(self.beta)

        q_values = self.q_network(states)
        q_values_next = self.target_network(next_states)

        target_q_values = rewards + (self.gamma * q_values_next.max(1)[0] * (1 - dones))

        td_errors = target_q_values - q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

        new_priorities = td_errors.abs().detach().cpu().numpy() + 1e-6
        self.replay_buffer.update_priorities(indices, new_priorities)

        loss = (td_errors ** 2 * weights).mean()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.tau = min(1.0, self.tau + 0.01)
        for target_param, local_param in zip(self.target_network.parameters(), self.q_network.parameters()):
            target_param.data.copy_(self.tau * local_param.data + (1.0 - self.tau) * target_param.data)

    def save(self, path):
        torch.save(self.q_network.state_dict(), path)

    def load(self, path):
        self.q_network.load_state_dict(torch.load(path))
        self.target_network.load_state_dict(self.q_network.state_dict())

# Warehouse Environment
class WarehouseEnv(gym.Env):
    def __init__(self, grid, num_autobots, max_steps=200):
        super(WarehouseEnv, self).__init__()
        self.grid = grid
        self.num_autobots = num_autobots
        self.max_steps = max_steps
        self.current_step = 0
        self.commands = defaultdict(int)

        self.autobot_start = {}
        self.autobot_end = {}
        self.find_start_end_positions(['A1', 'A2'], ['B1', 'B2'])

        self.obstacles = set()
        self.find_obstacles(['X'])

        self.moving_obstacles = set()
        self.find_moving_obstacles(['M'])

        self.action_space = gym.spaces.MultiDiscrete([5] * num_autobots)
        self.observation_space = gym.spaces.Box(
            low=0, high=max(len(grid), len(grid[0])),
            shape=(num_autobots, 8), dtype=np.float32
        )

        self.autobot_positions = self.autobot_start.copy()
        self.done = [False] * self.num_autobots

    def find_start_end_positions(self, start_symbols, end_symbols):
        for agent_id, symbol in enumerate(start_symbols):
            found = False
            for i, row in enumerate(self.grid):
                for j, cell in enumerate(row):
                    if cell == symbol:
                        self.autobot_start[agent_id] = (i, j)
                        found = True
                        break
                if found:
                    break
            if not found:
                raise ValueError(f"Start symbol {symbol} not found in grid.")

        for agent_id, symbol in enumerate(end_symbols):
            found = False
            for i, row in enumerate(self.grid):
                for j, cell in enumerate(row):
                    if cell == symbol:
                        self.autobot_end[agent_id] = (i, j)
                        found = True
                        break
                if found:
                    break
            if not found:
                raise ValueError(f"End symbol {symbol} not found in grid.")

    def find_obstacles(self, obstacle_symbols):
        for symbol in obstacle_symbols:
            for i, row in enumerate(self.grid):
                for j, cell in enumerate(row):
                    if cell == symbol:
                        self.obstacles.add((i, j))

    def find_moving_obstacles(self, moving_obstacle_symbols):
        for symbol in moving_obstacle_symbols:
            for i, row in enumerate(self.grid):
                for j, cell in enumerate(row):
                    if cell == symbol:
                        self.moving_obstacles.add((i, j))

    def reset(self):
        self.current_step = 0
        self.autobot_positions = self.autobot_start.copy()
        self.done = [False] * self.num_autobots
        return self.get_observation()

    def get_observation(self):
        obs = []
        for agent_id in range(self.num_autobots):
            x, y = self.autobot_positions[agent_id]
            end_x, end_y = self.autobot_end[agent_id]
            obs.append([x, y, end_x, end_y,
                        self.commands[agent_id],
                        self.done[agent_id],
                        len(self.obstacles),
                        len(self.moving_obstacles)])
        return np.array(obs, dtype=np.float32)

    def step(self, actions):
        for agent_id in range(self.num_autobots):
            if not self.done[agent_id]:
                self.commands[agent_id] += 1
                command = actions[agent_id]

                if command == 0:  # Forward
                    self.move_autobot(agent_id, 0, 1)
                elif command == 1:  # Backward
                    self.move_autobot(agent_id, 0, -1)
                elif command == 2:  # Left
                    self.move_autobot(agent_id, -1, 0)
                elif command == 3:  # Right
                    self.move_autobot(agent_id, 1, 0)
                elif command == 4:  # Wait
                    continue

        self.current_step += 1
        done = all(self.done) or self.current_step >= self.max_steps
        reward = self.get_reward(done)

        return self.get_observation(), reward, done, {}

    def move_autobot(self, agent_id, dx, dy):
        x, y = self.autobot_positions[agent_id]
        new_x, new_y = x + dx, y + dy
        if (0 <= new_x < len(self.grid) and
                0 <= new_y < len(self.grid[0]) and
                (new_x, new_y) not in self.obstacles):
            self.autobot_positions[agent_id] = (new_x, new_y)

        if self.autobot_positions[agent_id] == self.autobot_end[agent_id]:
            self.done[agent_id] = True

    def get_reward(self, done):
        if done and all(self.done):
            return 100
        elif done:
            return -50
        else:
            return -1

# Main Function to Train Agents
def main():
    grid = [
        ['.', '.', '.', 'A1', '.', 'X'],
        ['.', 'X', '.', '.', '.', '.'],
        ['A2', '.', '.', 'X', '.', 'B1'],
        ['.', '.', 'X', '.', '.', '.'],
        ['B2', '.', '.', '.', '.', '.'],
    ]

    num_autobots = 2
    env = WarehouseEnv(grid, num_autobots)

    agents = [IQLAgent(state_size=8, action_size=5, agent_id=i) for i in range(num_autobots)]

    num_episodes = 1000
    for episode in range(num_episodes):
        states = env.reset()
        episode_reward = 0
        done = False

        while not done:
            actions = [agent.choose_action(states[i]) for i, agent in enumerate(agents)]
            next_states, rewards, done, _ = env.step(actions)
            episode_reward += rewards

            for i, agent in enumerate(agents):
                agent.step(states[i], actions[i], rewards, next_states[i], done)

            states = next_states

        print(f"Episode: {episode + 1}, Epsilon: {agents[0].epsilon:.2f}, Reward: {episode_reward:.2f}")

    # Save trained models   
    os.makedirs("trained_models", exist_ok=True)
    for i, agent in enumerate(agents):
        agent.save(f"trained_models/agent_{i}.pth")

if __name__ == "__main__":
    main()